# Análise Geográfica dos Times da NBA
Esta página permite explorar a distribuição geográfica dos times e o total de pontos por cidade e por data dos jogos.

In [2]:
import pandas as pd
import panel as pn
import plotly.express as px
from datetime import datetime
pn.extension('plotly')
# Carregar dados
team = pd.read_csv('csv/team.csv')
game = pd.read_csv('csv/game.csv', parse_dates=['game_date'])
game_info = pd.read_csv('csv/game_info.csv', parse_dates=['game_date'])
# Garantir colunas minúsculas
team.columns = [c.lower() for c in team.columns]
game.columns = [c.lower() for c in game.columns]
game_info.columns = [c.lower() for c in game_info.columns]
# Carregar coordenadas das cidades (exemplo: pode ser necessário um dataset externo, aqui simulado)
# Supondo que team.csv tem 'city', 'latitude', 'longitude'
if 'latitude' not in team.columns or 'longitude' not in team.columns:
    # Simulação: adicionar coordenadas fictícias para cidades únicas
    import numpy as np
    cities = team['city'].unique()
    coords = pd.DataFrame({'city': cities, 'latitude': np.random.uniform(25, 50, len(cities)), 'longitude': np.random.uniform(-125, -70, len(cities))})
    team = pd.merge(team, coords, on='city', how='left')

In [ ]:
# Filtro de cidade
cities = sorted(team['city'].dropna().unique())
city_selector = pn.widgets.MultiSelect(name='Selecione a Cidade', options=cities, value=[cities[0]], size=8)
# Agrupar pontos por cidade
def pontos_por_cidade(selected_cities):
    teams_cidade = team[team['city'].isin(selected_cities)]
    df = pd.merge(teams_cidade[['id', 'city', 'latitude', 'longitude']], game, left_on='id', right_on='team_id_home', how='left')
    pontos = df.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
    fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', hover_name='city', size_max=60, zoom=3, mapbox_style='open-street-map', title='Total de Pontos por Cidade (Home)')
    fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

In [ ]:
# Filtro de data (usando game_info)
min_date = game_info['game_date'].min().date()
max_date = game_info['game_date'].max().date()
date_slider = pn.widgets.DateRangeSlider(name='Intervalo de Data dos Jogos', start=min_date, end=max_date, value=(min_date, max_date))
def pontos_por_data(date_range):
    # game_info não possui team_id_home, então precisamos usar game.csv para obter os times e pontos
    # Merge game_info com game.csv para obter os times de cada jogo
    df = pd.merge(game_info, game[['game_id','team_id_home','pts_home']], on='game_id', how='left')
    df = pd.merge(df, team[['id', 'city', 'latitude', 'longitude']], left_on='team_id_home', right_on='id', how='left')
    df = df[(df['game_date'] >= pd.to_datetime(date_range[0])) & (df['game_date'] <= pd.to_datetime(date_range[1]))]
    pontos = df.groupby(['city', 'latitude', 'longitude']).agg({'pts_home': 'sum'}).reset_index()
    fig = px.scatter_mapbox(pontos, lat='latitude', lon='longitude', size='pts_home', color='pts_home', hover_name='city', size_max=60, zoom=3, mapbox_style='open-street-map', title='Total de Pontos por Cidade (Filtrado por Data)')
    fig.update_layout(height=650, margin=dict(l=0, r=0, t=40, b=0))
    return pn.pane.Plotly(fig, config={'displayModeBar': True, 'scrollZoom': True}, height=650)

In [7]:
# Layout final
geo_dashboard = pn.Column(
    pn.pane.Markdown('## Análise Geográfica'),
    pn.Row(
        pn.Column(
            pn.pane.Markdown('### Filtro por Cidade'),
            city_selector,
            pn.bind(pontos_por_cidade, city_selector),
        ),
        pn.Column(
            pn.pane.Markdown('### Filtro por Data dos Jogos'),
            date_slider,
            pn.bind(pontos_por_data, date_slider)
        ),
    ),
)

geo_dashboard

/tmp/ipykernel_83331/1446286621.py:10: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/tmp/ipykernel_83331/3080393987.py:12: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

/tmp/ipykernel_83331/3080393987.py:12: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

